In [1]:
import pandas as pd
import numpy as np
from numpy import argmax
from numpy import sqrt
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.utils import resample
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from matplotlib import pyplot
from statistics import median
import pickle
import csv
import warnings
import datetime
import multiprocessing
warnings.filterwarnings("ignore")

In [2]:
project_list = ['geoserver', 'gradle', 'cloud_controller_ng', 'opal', 'jruby', 'cloudify', 'chef', 'orbeon-forms', 'vagrant']

In [3]:
def output_values(Y_data):
    Y_t = []
    for e in Y_data:
        if e == 'passed':
            Y_t.append(1)
        else:
            Y_t.append(0) 
    return Y_t

In [4]:
def get_pass_streak(y_project):
    p = y_project[0]
    pass_streak = [y_project[0]]
    for i in range(1, len(y_project)):
        pass_streak.append(p)
        if y_project[i] == 1:
            p += 1
        else:
            p = 0
    return pass_streak

In [5]:
def get_first_failures(df):
    
    results = df['tr_status'].tolist()
    length = len(results)
    verdict = ['keep']
    prev = results[0]
    
    for i in range(1, length):
        if results[i] == 0:
            if prev == 0:
                verdict.append('discard')
                #print(i+1)
            else:
                verdict.append('keep')
        else:
            verdict.append('keep')
        prev = results[i]
    
    df['verdict'] = verdict
    df = df[ df['verdict'] == 'keep' ]
    df.drop('verdict', inplace=True, axis=1)
    return df

In [6]:
def get_complete_data(p_name):
    
    #open the metrics file
    filename = 'metrics_data/' + p_name + '_metrics.csv'
    project = pd.read_csv(filename)
    
    #clean the data & remove correlated columns
    project = project [ project['developer_experience'] >= 0]
    project.drop('num_commits', inplace=True, axis=1)
    project.drop('reviewer_experience', inplace=True, axis=1)
    project.drop('num_of_reviewers', inplace=True, axis=1)
    
    build_ids = project['tr_build_id'].tolist()
    #get results data
    res_file = '../data/' + p_name + '.csv'
    res_project = pd.read_csv(res_file, usecols = ['tr_build_id', 'gh_build_started_at', 'tr_status'])
    res_project['gh_build_started_at'] =  pd.to_datetime(res_project['gh_build_started_at'], format='%Y-%m-%d %H:%M:%S')
    y_project = res_project[res_project['tr_build_id'].isin(build_ids)]['tr_status'].tolist()
    y_project = output_values(y_project)
    
    #append date of build
    project_dates = res_project[res_project['tr_build_id'].isin(build_ids)]['gh_build_started_at'].tolist()
    project['gh_build_started_at'] = project_dates
    
    #add results column to the dataframe
    project['tr_status'] = y_project
    
    return project

In [7]:
def get_start_end_date(project):
    dates = project['gh_build_started_at'].tolist()
    
    start_date = dates[0] - datetime.timedelta(days = 1)
    end_date = dates[-1] - datetime.timedelta(days = 1)
    
    return start_date, end_date

In [8]:
def get_required_data(p_name, build_ids):
    
    res_file = '../data/' + p_name + '.csv'
    res_project = pd.read_csv(res_file, usecols = ['tr_build_id', 'tr_duration'])
    durations = res_project[res_project['tr_build_id'].isin(build_ids)]['tr_duration'].tolist()
    return durations

In [15]:
def compute_performance(p_name, test_builds, test_result, pred_result):
    
    
    
    durations = get_required_data(p_name, test_builds)
    actual_duration = sum(durations)
    actual_failures = test_result.count(0)
    
    total_builds = len(test_builds)
    num_of_builds = 0
    total_duration = 0
    cbf = 0
    saved_builds = 0
    
    batch = []
    batch_duration = []
    actual_results = []
    max_batch_size = 4
    
    for i in range(len(pred_result)):
        if pred_result[i] == 0:
            
            if test_result[i] == 0:
                cbf += 1
                
            if len(batch) < max_batch_size:
                batch.append(pred_result[i])
                batch_duration.append(durations[i])
                actual_results.append(test_result[i])
            
            if len(batch) == max_batch_size:
                num_of_builds += 1
                total_duration += max(batch_duration)
                
                if 0 in actual_results:
                    num_of_builds += 4
                    total_duration += sum(batch_duration)
        else:
            saved_builds += 1
            
    if len(batch) > 0:
        num_of_builds += 1
        total_duration += max(batch_duration)
        
        if 0 in actual_results:
            num_of_builds += len(batch)
            total_duration += sum(batch_duration)
                    
    #Delay computation
    flag = 0
    count = 0
    delay = []
    for i in range(len(pred_result)):
        if flag == 1:
            if pred_result[i] == 1:
                count += 1
            
            if pred_result[i] == 0:
                delay.append(count)
                count = 0
                flag = 0
                
        if test_result[i] != 1:
            if pred_result[i] == 1:
                flag = 1
    delay.append(count)

    
    try:
        
        time_saved = 100*total_duration/actual_duration
        builds_saved = 100*saved_builds/total_builds
        reqd_builds = 100*num_of_builds/total_builds
        failed = 100*cbf/actual_failures
        median_delays = median(delay)
        total_delays = sum(delay)
    
#         print("===========================================")
#         print('The performance of the model is as follows:')
#         print('\t Time Reqd : {}'.format(total_duration))
#         print('\t % Time Reqd : {}%'.format(time_saved))
#         print('\t Num. Builds saved : {}%'.format(saved_builds))
#         print('\t % Builds saved : {}%'.format(builds_saved))
#         print('\t Num. Builds required : {}'.format(num_of_builds))
#         print('\t % Builds required : {}%'.format(reqd_builds))
#         print('\t Num. Failed Builds Identified : {}'.format(cbf))
#         print('\t % Failed Builds Identified : {}%'.format(failed))
#         print('\t Median Delay Induced : {} builds'.format(median_delays))
#         print('\t Total Delay Induced: {} builds'.format(total_delays))
#         print('\t Total number of builds: {}'.format(total_builds))
#         print('\t Total number of failed builds: {}'.format(actual_failures))
#         print('\t Total Duration: {}'.format(actual_duration))
#         print("===========================================")
        
    except:
        
#         print('exception')
        return (0, 0, 0, 0, 0, 0)
    
    return (time_saved, builds_saved, reqd_builds, failed, median_delays, total_delays)
    
    

In [ ]:
def bootstrapping(p_name):
    
    performances = {'time_saved':[], 'builds_saved':[], 'builds_reqd':[], 'failed_builds':[], 'total_delay':[], 'median_delay':[]}
#     print('Processing {}'.format(p_name))
    
    project = get_complete_data(p_name)
    start_date, end_date = get_start_end_date(project)
    
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    max_depth = [int(x) for x in np.linspace(10, 110, num = 5)]
    
    param_grid = {'n_estimators': n_estimators, 'max_depth': max_depth}
    forest = RandomForestClassifier()
    grid_search = GridSearchCV(estimator = forest, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 0)
    
    phase = 1

    while start_date < end_date:
        
        train_period = 100
        test_period = 10
        
        while True:
            train_end = start_date + datetime.timedelta(days = train_period + 1)
            test_start = start_date + datetime.timedelta(days = train_period)
            test_end = test_start + datetime.timedelta(days = test_period)

            #getting data of train & test phase wise
            train_data = project[ (project['gh_build_started_at'] > start_date) & (project['gh_build_started_at'] < train_end)]
            test_data = project[ (project['gh_build_started_at'] > test_start) & (project['gh_build_started_at'] < test_end)]

            #getting 'y' data
            train_result = train_data['tr_status'].tolist()
            test_result = test_data['tr_status'].tolist()
            
            if len(train_result) > 100 and len(test_result) > 10 :
                break
            
            if len(train_result) <= 100:
                train_period += 20
            
            if len(test_result) <= 10:
                test_period += 20
                
            
        
        #dropping build start time column
        train_data.drop('gh_build_started_at', inplace=True, axis=1)
        test_data.drop('gh_build_started_at', inplace=True, axis=1)
        
        #add pass_streak to training data:
        train_data['num_of_passes'] = get_pass_streak(train_result)
        
        best_n_estimators = []
        best_max_depth = []
        
        best_f1 = 0
        best_f1_sample = 0
        best_f1_sample_result = 0
        best_f1_estimator = 0
        best_thresholds = []
        
#         train_result = train_data['tr_status']
        
#         train_data.drop('tr_status', inplace=True, axis=1)
#         train_data.drop('tr_status', inplace=True, axis=1)
        
#         test_data.drop('tr_build_id', inplace=True, axis=1)
#         test_data.drop('tr_build_id', inplace=True, axis=1)
        
#         grid_search.fit(train_data, train_result)
        
        #bootstrap 10 times
        for i in range(2):
            
            file_name = 'rq2_' + p_name + '_' + str(phase) + '_model_' + str(i+1) + '_model.pkl'
            sample_train = resample(train_data, replace=True, n_samples=len(train_data))
            sample_train_result = sample_train['tr_status']
            
            build_ids = sample_train['tr_build_id'].tolist()
            sample_test = train_data [~train_data['tr_build_id'].isin(build_ids)] 
            sample_test_result = sample_test['tr_status']
            
            #dropping result column and build ids column
            sample_train.drop('tr_status', inplace=True, axis=1)
            sample_train.drop('tr_build_id', inplace=True, axis=1)
            sample_test.drop('tr_status', inplace=True, axis=1)
            sample_test.drop('tr_build_id', inplace=True, axis=1)
            
            #training
            grid_search.fit(sample_train, sample_train_result)
            sample_pred_vals = grid_search.predict_proba(sample_test)
            
            pred_vals = sample_pred_vals[:, 1]
            fpr, tpr, t = roc_curve(sample_test_result, pred_vals)
            gmeans = sqrt(tpr * (1-fpr))
            ix = argmax(gmeans)
            bt = t[ix]
            best_thresholds.append(bt)
            
            final_pred_result = []
            #threshold setting
            for j in range(len(pred_vals)):
                if pred_vals[j] > bt:
                    final_pred_result.append(1)
                else:
                    final_pred_result.append(0)
            
            try:
                accuracy = accuracy_score(sample_test_result, final_pred_result)
                precision = precision_score(sample_test_result, final_pred_result)
                recall = recall_score(sample_test_result, final_pred_result)
                confusion = confusion_matrix(sample_test_result, final_pred_result)
                auc_score = roc_auc_score(sample_test_result, final_pred_result)
                f1 = f1_score(sample_test_result, final_pred_result)
            except:
                print('')
    
            if f1 > best_f1:
                best_f1 = f1
                best_f1_sample = sample_train
                best_f1_sample_result = sample_train_result
                best_f1_estimator = grid_search.best_estimator_

#             print(precision, recall, accuracy, f1, auc_score)
            best_n_estimators.append(grid_search.best_params_['n_estimators'])
            best_max_depth.append(grid_search.best_params_['max_depth'])
        
        #completed with bootstrapping 
        threshold = median(best_thresholds)
        n_estimator = median(best_n_estimators)
        max_depth = median(best_max_depth)
        #retrain on the best 
        forest = RandomForestClassifier(n_estimators=int(n_estimator), max_depth=int(max_depth))
        forest.fit(best_f1_sample, best_f1_sample_result)
        
        test_builds = test_data['tr_build_id'].tolist()
        test_data.drop('tr_build_id', inplace=True, axis=1)
        test_data.drop('tr_status', inplace=True, axis=1)
        
        final_pred_result = []
        queue = 0
        i = 0
        total = len(test_data)
        while i < total :
            data = test_data.iloc[i]
            data['num_of_passes'] = queue
            predict = forest.predict_proba([data])
            if predict[0][1] > threshold:
                final_pred_result.append(1)
                queue += 1
                i+=1
            else:
                final_pred_result.append(0)
                queue = 0
                i += 1
                
                
#         print('Individual testing for {}....'.format(p_name))
        
        try:
            accuracy = accuracy_score(test_result, final_pred_result)
            precision = precision_score(test_result, final_pred_result)
            recall = recall_score(test_result, final_pred_result)
            confusion = confusion_matrix(test_result, final_pred_result)
            auc_score = roc_auc_score(test_result, final_pred_result)
            f1 = f1_score(test_result, final_pred_result)

#             print(precision, recall, accuracy, f1, auc_score)
#             print(confusion)
            
        except:
            print('')
        
        batch_performance = compute_performance(p_name, test_builds, test_result, final_pred_result)
        
        performances['time_saved'].append(batch_performance[0])
        performances['builds_saved'].append(batch_performance[1])
        performances['builds_reqd'].append(batch_performance[2])
        performances['failed_builds'].append(batch_performance[3])
        performances['median_delay'].append(batch_performance[4])
        performances['total_delay'].append(batch_performance[5])
        
        start_date = test_end
        phase += 1
    
    
    #Project Performance:
    
    print("Average Time Saved in {} = {}".format(p_name, sum(performances['time_saved'])/len(performances['time_saved'])))
    print("Average Builds Saved in {} = {}".format(p_name, sum(performances['builds_saved'])/len(performances['builds_saved'])))
    print("Average Builds Reqd in {} = {}".format(p_name, sum(performances['builds_reqd'])/len(performances['builds_reqd'])))
    print("Average Failed Identified in {} = {}".format(p_name, sum(performances['failed_builds'])/len(performances['failed_builds'])))
    print("Average Median Delay in {} = {}".format(p_name, sum(performances['median_delay'])/len(performances['median_delay'])))
    print("Average Total Delay in {} = {}".format(p_name, sum(performances['total_delay'])/len(performances['total_delay'])))
    

In [ ]:
for p_name in project_list:
    

    print('\n\n\n\n\n')

Processing geoserver
<class 'numpy.ndarray'>
[0.04947222 0.11111111 0.15388889 0.42766402 0.78722222 0.54888889
 0.37444444 0.67222222 0.26671296 0.04947222 0.01350926 0.31482407
 0.06884259 0.42273148 0.34555556 0.89713492 0.01350926 0.27518519
 0.86244444 0.97496429 0.78722222 0.165      0.83611111 0.81381481
 0.91093519 0.94380556 0.3706541  0.37131504 0.73888889 0.47293254
 0.38368056 0.36359259 0.07277778 0.93538889 0.99362169 0.04333333
 0.57666667 0.93277778 0.98010317 0.98690873 0.99       0.98444444
 0.95777778 0.99444444 0.56111111 0.49388889 0.91166667 0.57055556
 0.90833333 0.88388889 0.53833333 0.355      0.3751034  0.60833333
 0.67333333 0.60643519 0.36605093 0.29222222 0.31555556 0.555
 0.365      0.21777778 0.71333333 0.45962963 0.26666667 0.41
 0.9110754  0.42       0.24388889 0.58833333 0.34833333 0.13222222
 0.12055556 0.36106481 0.59722222 0.36944444 0.68611111]
0.5294117647058824 0.4864864864864865 0.5454545454545454 0.5070422535211269 0.5432432432432432
<class 'nu